<a href="https://colab.research.google.com/github/liuchiente/code_from_colab_and_kaggle/blob/main/Crew_AI_with_AI_Agent_By_LiuChien_(DeepSeek)_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crew AI with AI Angent By LiuChien (DeepSeek)
Crew AI是一套Python函式庫，讓你可以輕鬆實現AI Agent，如果想搭配更多樣化的LLM，也可以搭配LiteLM。

這個筆記本的程式雖然不多，但卻簡單實現了將一個 Ionic Project從Git下載之後進行解析，並且依照專案架構和相依套件進行解析，判斷目前專案出有什麼重大問題。

---

接下來我們採用了幾個步驟，為一個專案做健康檢查：


1. 下載專案
2. 分析專案架構
3. 設定Crew AI、LiteLM (這裡設定用DeepSeek模型)
4. 設定Agent (包含結構專家、依賴判斷專家、重構建議)
5. 設定任務 (分別進行結構、依賴判斷，並給出建議)
6. 開始解析並產出結果


---


參考資料

* https://www.crewai.com
* https://docs.litellm.ai
* https://docs.litellm.ai/docs/providers
* https://aistudio.google.com/u/4/api-keys?hl=zh-tw




# 安裝Crew AI與其他工具
1. Langchain
2. CrewAI
3. OpenAI
4. LiteLM



In [1]:
# 安裝 CrewAI 與其他工具
!pip install rich networkx matplotlib
!pip install -U crewai openai
!pip install langchain langchain-community
!pip install litellm
!pip install --upgrade crewai litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.3/576.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.

# 設定金鑰


1.   HuggingFace金鑰 (給LiteLM使用)
2.   DeepSeek金鑰

因為我們會用到LiteLM串接HuggingFace上的LLM模型，因此一定要設定。



In [1]:
import os
import litellm

# 使用你的DeepSeek API金鑰
os.environ['DEEPSEEK_API_KEY']="sk-fxxxxxxxxxxxxxxx9"
os.environ["HUGGINGFACE_API_KEY"] = "hf_xxxxxxxxxxxxxxxxxxxxIo"

# 如果套件都成功的話,應該要印出正確路徑
print("LiteLLM 模塊的路徑是:")
print(litellm.__file__)

LiteLLM 模塊的路徑是:
/usr/local/lib/python3.12/dist-packages/litellm/__init__.py


# 示範用的Github專案


1.   一個年久失修，Ionic V1版本為基礎的Hybrid APP專案


In [2]:
# GitHub 專案網址
github_url = "https://github.com/liuchiente/fonleeEnterprise.git"
project_name = github_url.split("/")[-1].replace(".git", "")
project_path = f"/content/{project_name}"

# Clone 專案
!git clone {github_url}

fatal: destination path 'fonleeEnterprise' already exists and is not an empty directory.


# 掃描專案架構


1.   專案架構和目錄組成
2.   相依套件組成



In [5]:
import os, json
from rich.tree import Tree
from rich import print
import networkx as nx
import matplotlib.pyplot as plt

def build_tree(path, tree=None):
    if tree is None:
        tree = Tree(f"[bold blue]{os.path.basename(path)}[/]")
    for item in sorted(os.listdir(path)):
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            subtree = tree.add(f"[bold green]{item}/[/]")
            build_tree(full_path, subtree)
        else:
            tree.add(item)
    return tree

# 解析資料夾結構
print("[bold underline]專案資料夾結構：[/]")
tree = build_tree(project_path)
print(tree)

# 解析 package.json
print("[bold underline]依賴套件結構：[/]")
package_path = os.path.join(project_path, "package.json")
dependencies = {}
if os.path.exists(package_path):
    with open(package_path, "r") as f:
        package_data = json.load(f)
    dependencies = package_data.get("dependencies", {})
    for dep, version in dependencies.items():
        print(f"- {dep}: {version}")


專案資料夾結構：

fonleeEnterprise
├── .bowerrc
├── .editorconfig
├── .git/
│   ├── HEAD
│   ├── branches/
│   ├── config
│   ├── description
│   ├── hooks/
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── push-to-checkout.sample
│   │   └── update.sample
│   ├── index
│   ├── info/
│   │   └── exclude
│   ├── logs/
│   │   ├── HEAD
│   │   └── refs/
│   │       ├── heads/
│   │       │   └── master
│   │       └── remotes/
│   │           └── origin/
│   │               └── HEAD
│   ├── objects/
│   │   ├── info/
│   │   └── pack/
│   │       ├── pack-db918074c26a04862dc252dadb005a3a350609c3.idx
│   │       └── pack-db918074c26a04862dc252dadb005a3a350609c3.pack
│   ├── packed-refs
│   └── refs/
│       ├── heads/
│       │   └── master
│       ├── remotes/
│       │   └── origin/
│       │       └── HEAD
│       └── tags/
├── .gitignore
├── GoogleService-Info.plist
├── bower.json
├── build.json
├── config.xml
├── google-services.json
├── gulpfile.js
├── hooks/
│   ├── README.md
│   └── after_prepare/
│       └── 010_add_platform_class.js
├── ionic.config.json
├── ionic.starter.json
├── network_security_config.xml
├── package-lock.json
├── package.json
├── res/
│   ├── drawable-land-hdpi/
│   │   └── screen.png
│   ├── drawable-land-ldpi/
│   │   └── screen.png
│   ├── drawable-land-mdpi/
│   │   └── screen.png
│   ├── drawable-land-xhdpi/
│   │   └── screen.png
│   ├── drawable-land-xxhdpi/
│   │   └── screen.png
│   ├── drawable-land-xxxhdpi/
│   │   └── screen.png
│   ├── drawable-port-hdpi/
│   │   └── screen.png
│   ├── drawable-port-ldpi/
│   │   └── screen.png
│   ├── drawable-port-mdpi/
│   │   └── screen.png
│   ├── drawable-port-xhdpi/
│   │   └── screen.png
│   ├── drawable-port-xxhdpi/
│   │   └── screen.png
│   ├── drawable-port-xxxhdpi/
│   │   └── screen.png
│   ├── mipmap-hdpi/
│   │   └── icon.png
│   ├── mipmap-ldpi/
│   │   └── icon.png
│   ├── mipmap-mdpi/
│   │   └── icon.png
│   ├── mipmap-xhdpi/
│   │   └── icon.png
│   ├── mipmap-xxhdpi/
│   │   └── icon.png
│   └── mipmap-xxxhdpi/
│       └── icon.png
├── resources/
│   ├── android/
│   │   ├── icon/
│   │   │   ├── drawable-hdpi-icon.png
│   │   │   ├── drawable-ldpi-icon.png
│   │   │   ├── drawable-mdpi-icon.png
│   │   │   ├── drawable-xhdpi-icon.png
│   │   │   ├── drawable-xxhdpi-icon.png
│   │   │   ├── drawable-xxhdpi.png
│   │   │   └── drawable-xxxhdpi-icon.png
│   │   └── splash/
│   │       ├── drawable-land-hdpi-screen.png
│   │       ├── drawable-land-ldpi-screen.png
│   │       ├── drawable-land-mdpi-screen.png
│   │       ├── drawable-land-screen.png
│   │       ├── drawable-land-xhdpi-screen.png
│   │       ├── drawable-land-xxhdpi-screen.png
│   │       ├── drawable-land-xxxhdpi-screen.png
│   │       ├── drawable-port-hdpi-screen.png
│   │       ├── drawable-port-ldpi-screen.png
│   │       ├── drawable-port-mdpi-screen.png
│   │       ├── drawable-port-screen.png
│   │       ├── drawable-port-xhdpi-screen.png
│   │       ├── drawable-port-xxhdpi-screen.png
│   │       └── drawable-port-xxxhdpi-screen.png
│   ├── icon.png
│   ├── icon.png.md5
│   ├── ios/
│   │   ├── icon/
│   │   │   ├── icon-1024.png
│   │   │   ├── icon-20.png
│   │   │   ├── icon-24@2x.png
│   │   │   ├── icon-27.5@2x.png
│   │   │   ├── icon-40.png
│   │   │   ├── icon-40@2x.png
│   │   │   ├── icon-40@3x.png
│   │   │   ├── icon-44@2x.png
│   │   │   ├── icon-50.png
│   │   │   ├── icon-50@2x.png
│   │   │   ├── icon-60.png
│   │   │   ├── icon-60@2x.png
│   │   │   ├── icon-60@3x.png
│   │   │   ├── icon-72.png
│   │   │   ├── icon-72@2x.png
│   │   │   ├── icon-76.png
│   │   │   ├── icon-76@2x.png
│   │   │   ├── icon-83.5@2x.png
│   │   │   ├── icon-86@2x.pn

依賴套件結構：

- @ionic/angular: ^4.11.13

- @ionic/cli: ^7.2.0

...

- gulp: ^3.9.1

- gulp-concat: ^2.2.0

- gulp-minify-css: ^0.3.0

- gulp-rename: ^1.2.0

- ionic-native: ^2.9.0

# 定義Agent並指定LLM模型

依照以下目標，我們對每個Agent設定了角色、工作目標、背景知識，必要的話也可以切換不同LLM模型一起協作。

1.   結構分析
2.   依賴分析
3.   重構建議

特別注意我們這裡用的是LiteLM，串接HuggingFace上的Text-Generation模型，所以這裡傳入的是指定型位置的字串。
想知道有哪些LLM可以用，可以參閱https://docs.litellm.ai/docs/providers。



In [3]:
from crewai import Crew, Agent, Task
from langchain_community.llms import HuggingFaceHub

# Agent 1：結構分析
structure_agent = Agent(
    role="Structure Analyst",
    goal="分析專案資料夾結構並找出模組邏輯",
    backstory="你是資深架構師，擅長從目錄結構判斷程式設計模式。",
    llm="huggingface/together/deepseek-ai/DeepSeek-R1"  # 透過 LiteLLM 自動處理
)

# Agent 2：依賴分析
dependency_agent = Agent(
    role="Dependency Analyst",
    goal="解析 package.json 並找出潛在相依問題",
    backstory="你是精通AngularJS,Ionic,Javascript的專家，熟悉各種套件版本與相容性。",
    llm="huggingface/together/deepseek-ai/DeepSeek-R1"
)

# Agent 3：重構建議
refactor_agent = Agent(
    role="Refactor Strategist",
    goal="根據結構與依賴分析，提出重構建議",
    backstory="你是專案顧問，專精於模組化與技術債清理。",
    llm="huggingface/together/deepseek-ai/DeepSeek-R1"
)

# 定義各Agent需要執行的任務

前兩個任務分別是分析專案結構、套件相依問題，並且可以指定任務，第三個任務是重構建議，則可以直接請Agent用前兩個任務總結下去做建議，Crew AI會自動幫你處理上下文，想像一下你就是在一個聊天視窗，分別作以下三個任務的行為。

In [6]:
# 將掃描結果轉為文字輸入
structure_text = tree.__str__()
dependency_text = "\n".join([f"{k}: {v}" for k, v in dependencies.items()])

task1 = Task(
    description=f"根據以下資料夾結構分析模組邏輯：\n{structure_text}",
    expected_output="列出專案的主要模組與其功能定位",
    agent=structure_agent
)

task2 = Task(
    description=f"根據以下 dependencies 分析相依問題：\n{dependency_text}",
    expected_output="指出可能的版本衝突、過時套件或安全性風險",
    agent=dependency_agent
)

task3 = Task(
    description="請根據前兩項分析結果，提出重構建議與模組優化方向。",
    expected_output="提供具體重構建議與模組拆分策略",
    agent=refactor_agent
)

# 建立一個 CrewAI 的團隊（Crew）物件
crew = Crew(
    agents=[structure_agent, dependency_agent, refactor_agent],  # 指定三位 Agent，各自負責不同分析任務
    tasks=[task1, task2, task3],  # 指定三個Task，會依照順序分配給 Agent 執行
    verbose=True  # 啟用詳細模式，執行過程中會印出每個 Agent 的思考與回覆
)

# 啟動整個分析流程，讓 Agent 開始執行任務並協作產出結果
result = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Structure Analyst                                                                                       │
│                                                                                                                 │
│  Task: 根據以下資料夾結構分析模組邏輯：                                                                         │
│  <rich.tree.Tree object at 0x7db2702cddf0>                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a8d17f07-d1c2-4a5e-badf-73ac2b9b576b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Dependency Analyst                                                                                      │
│                                                                                                                 │
│  Task: 根據以下 dependencies 分析相依問題：                                                                     │
│  @ionic/angular: ^4.11.13                                                                                       │
│  @ionic/cli: ^7.2.0                                                                                             │
│  @ionic/core: ^7.8.6                                                                                            │
│  cordova-plugin-androidx: 3.0.0                                                                                 │
│  cordova-plugin-androidx-adapter: 1.1.3                                                                         │
│  cordova-plugin-device: ^1.1.7                                                                                  │
│  cordova-plugin-ionic-keyboard: 2.2.0                                                                           │
│  cordova-plugin-splashscreen: ^4.1.0                                                                            │
│  cordova-plugin-statusbar: 2.4.2                                                                                │
│  cordova-support-android-plugin: ^1.0.2                                                                         │
│  cordova-support-google-services: ^1.4.1                                                                        │
│  globalthis: ^1.0.2                                                                                             │
│  gulp: ^3.9.1                                                                                                   │
│  gulp-concat: ^2.2.0                                                                                            │
│  gulp-minify-css: ^0.3.0                                                                                        │
│  gulp-rename: ^1.2.0                                                                                            │
│  ionic-native: ^2.9.0                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ae53e091-5551-4b9b-9263-c0dfa917ceb3                                                                     │
│  Agent: Dependency Analyst                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Refactor Strategist                                                                                     │
│                                                                                                                 │
│  Task: 請根據前兩項分析結果，提出重構建議與模組優化方向。                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Refactor Strategist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### 重構建議與模組優化方向                                                                                     │
│  基於模組結構分析與相依性診斷，提出以下具體重構方案，分為**技術債清理**、**模組拆分優化**及**依賴關係改進**三   │
│  大方向：                                                                                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **一、技術債清理優先行動**                                                                                │
│  1. **Ionic 框架升級策略**                                                                                      │
│     - 執行分階段升級路徑：                                                                                      │
│       - Phase 1：將 `@ionic/angular` 升級至 v7.8.6（與現有 `@ionic/core` 版本匹配）                             │
│       - Phase 2：同步升級 Angular 至 v15+（Ionic 7 最低要求）                                                   │
│       - Phase 3：重寫生命週期鉤子（如 `ionViewDidEnter` 改用 Angular 的 `ngAfterViewInit`）                     │
│     - 風險緩解：                                                                                                │
│       - 建立 `legacy-compat` 過渡層，封裝舊版元件行為                                                           │
│       - 使用 CSS 變量重寫樣式（替代 Shadow DOM 依賴）                                                           │
│                                                                                                                 │
│  2. **Cordova 插件現代化**                                                                                      │
│     | 插件名稱                  |                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [8]:
from IPython.display import Markdown, display

display(Markdown(str(result)))


### 重構建議與模組優化方向  
基於模組結構分析與相依性診斷，提出以下具體重構方案，分為**技術債清理**、**模組拆分優化**及**依賴關係改進**三大方向：  

---

#### **一、技術債清理優先行動**  
1. **Ionic 框架升級策略**  
   - 執行分階段升級路徑：  
     - Phase 1：將 `@ionic/angular` 升級至 v7.8.6（與現有 `@ionic/core` 版本匹配）  
     - Phase 2：同步升級 Angular 至 v15+（Ionic 7 最低要求）  
     - Phase 3：重寫生命週期鉤子（如 `ionViewDidEnter` 改用 Angular 的 `ngAfterViewInit`）  
   - 風險緩解：  
     - 建立 `legacy-compat` 過渡層，封裝舊版元件行為  
     - 使用 CSS 變量重寫樣式（替代 Shadow DOM 依賴）  

2. **Cordova 插件現代化**  
   | 插件名稱                  |